A common KPI for any company is it's market share. The basic formula for the calculation is company sales / total market sales. One neat way to get the denominator automactically (ie total market sales) is to use the Australian Bureau of Statistics API. The below script as an example gets the total sales for the retail supermarket Australia wide.

https://www.abs.gov.au/ausstats/abs@.nsf/Lookup/1407.0.55.002Main+Features3User+Guide

In [8]:
# First we will use some open source libraries
from datetime import datetime
import requests
import pandas as pd
import pypyodbc 
import numpy as np
import xlsxwriter

# Get start and end period for API query using the datetime library
now = datetime.now()
end = str(now.year) + "-" + str(now.month).zfill(2)
start = str(now.year - 3) + "-" + str('01')

"Start: " + start,"End: " + end

('Start: 2018-01', 'End: 2021-05')

Using the date time library, we can get our start and last date to insert into the url. This could also be done manually, or via the input() function built into python, but for my purposes I've created a variable which gets todays year and month as the end, and the same month three years ago for the start.

In [11]:
# Using the URL from the ABS website to make the request. Note that parameters are fed into the URL.

url = 'http://stat.data.abs.gov.au/sdmx-json/data/RT/0.2.01.10.M/all?detail=Full&dimensionAtObservation=AllDimensions&startPeriod=' + start + '&endPeriod=' + end
data = requests.get(url).json()

data

{'header': {'id': '530bf19f-33a1-4a76-8c32-3d762013191e',
  'test': False,
  'prepared': '2021-05-13T12:16:13.0868292Z',
  'sender': {'id': 'ABS', 'name': 'Australian Bureau of Statistics'},
  'links': [{'href': 'http://stat.data.abs.gov.au:80/sdmx-json/data/RT/0.2.01.10.M/all?detail=Full&dimensionAtObservation=AllDimensions&startPeriod=2018-01&endPeriod=2021-05',
    'rel': 'request'}]},
 'dataSets': [{'action': 'Information',
   'observations': {'0:0:0:0:0:0': [9001.13, 0, None],
    '0:0:0:0:0:1': [8252.32, 0, None],
    '0:0:0:0:0:2': [9348.06, 0, None],
    '0:0:0:0:0:3': [8685.34, 0, None],
    '0:0:0:0:0:4': [8950.29, 0, None],
    '0:0:0:0:0:5': [8625.18, 0, None],
    '0:0:0:0:0:6': [8918.78, 0, None],
    '0:0:0:0:0:7': [9115.1, 0, None],
    '0:0:0:0:0:8': [8968.78, 0, None],
    '0:0:0:0:0:9': [9355.1, 0, None],
    '0:0:0:0:0:10': [9390.87, 0, None],
    '0:0:0:0:0:11': [10536.39, 0, None],
    '0:0:0:0:0:12': [9494.2, 0, None],
    '0:0:0:0:0:13': [8703.54, 0, None],
    

As you can see, we get a JSON file with our data set. Its a bit messy, but we can iterate through the JSON file with some very simple code.

In [13]:
# This picks up the data
ds = data['dataSets'][0]['observations']
df = pd.DataFrame.from_dict(ds, orient = 'index').reset_index()

# This picks up the month periods
ds1 = data['structure']['dimensions']['observation'][5]['values']
df1 = pd.DataFrame(ds1).reset_index()

df.head(5), df1.head(5)

(         index        0  1     2
 0  0:0:0:0:0:0  9001.13  0  None
 1  0:0:0:0:0:1  8252.32  0  None
 2  0:0:0:0:0:2  9348.06  0  None
 3  0:0:0:0:0:3  8685.34  0  None
 4  0:0:0:0:0:4  8950.29  0  None,
    index       id      name
 0      0  2018-01  Jan-2018
 1      1  2018-02  Feb-2018
 2      2  2018-03  Mar-2018
 3      3  2018-04  Apr-2018
 4      4  2018-05  May-2018)

Now that we have our data frames, we can now combine them using the pandas funcion concat on the index.

In [14]:
combined = pd.concat([df, df1.reindex(df1.index)], axis=1).drop(['index',1,2],axis=1)

combined.head(10)

,0,id,name
0,9001.13,2018-01,Jan-2018
1,8252.32,2018-02,Feb-2018
2,9348.06,2018-03,Mar-2018
3,8685.34,2018-04,Apr-2018
4,8950.29,2018-05,May-2018
5,8625.18,2018-06,Jun-2018
6,8918.78,2018-07,Jul-2018
7,9115.10,2018-08,Aug-2018
8,8968.78,2018-09,Sep-2018
9,9355.10,2018-10,Oct-2018
